In [18]:
import dataiku

client = dataiku.api_client()
project = client.get_project('EVERYTHING')

source_folder = project.get_managed_folder("iV6c3jWU")
target_folder = project.get_managed_folder("oQL0pTOm")

# target_folder will now contain a copy of all files that were in source_folder
future = source_folder.copy_to(target_folder)
future.wait_for_result()

[{'path': '/dubai.index', 'size': 25600045, 'lastModified': 1663916927000}]

In [4]:
import numpy as np

d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [5]:
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:]) 

True
100000
[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751733 7.207629  7.2511625]
 [0.        6.3235645 6.684581  6.7999454]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279865 7.6628466]
 [0.        6.7638035 7.2951202 7.3688145]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


In [11]:
handle = dataiku.Folder('iV6c3jWU')
path = handle.get_path()
print(path)
path_in = os.path.join(path, 'dubai.index')
#faiss.write_index(index,path_in)    #### writing in filesystem folder
faiss.write_index(index,path_in)  # writing in local

/Users/rendo/Library/DataScienceStudio9/managed_folders/hmm/EVERYTHING/iV6c3jWU


In [16]:
import io

in_mem = io.BytesIO(index)

TypeError: a bytes-like object is required, not 'IndexFlatL2'

In [15]:
handle = dataiku.Folder("oQL0pTOm")
# This copies a local file to the managed folder
with open(in_mem) as f:
    handle.upload_stream("name_of_file_in_folder.index", f)
#handle.upload_file(path, index)

NameError: name 'in_mem' is not defined

In [22]:
target_folder = dataiku.Folder("oQL0pTOm")


In [31]:
with target_folder.get_download_stream("dubai.index") as stream:
    data = stream.readline()
    print(data)
    print(type(data))

b'IxF2@\x00\x00\x00\xa0\x86\x01\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x01\x01\x00\x00\x00\x00\xa8a\x00\x00\x00\x00\x00\xad\x1dD>\x85B\x1f?\xda\x1d\xe0>C\rI?\x7f\xacG?B\x91\x8b>\xb9\x8c\x8d>\x7fGM?\x9fHu?\x1f=`?\xd33\xb7>7A\x00?m\xf7.?\xa4s6?\x82\x91\xbd>\x8e\xaa\x0f?\x0f\xca\x00?\x10\x95a<\xf7\xd7E?\xc6\xf4a?V\xd2\xba>\x9b\x8a\x1d?{a\x9a=\x80\xd6\xbc>E\xe2n?\xb8\xc0&?#^\xcb>8\xeaI?X8\xa2>\xean\x11?"\x7f^?\x1fR\xdf>\x8cYM?\x9c7\x13>rJ4?q_4?\n'
<class 'bytes'>


In [34]:
faiss.read_index()

TypeError: Wrong number or type of arguments for overloaded function 'read_index'.
  Possible C/C++ prototypes are:
    faiss::read_index(char const *,int)
    faiss::read_index(char const *)
    faiss::read_index(FILE *,int)
    faiss::read_index(FILE *)
    faiss::read_index(faiss::IOReader *,int)
    faiss::read_index(faiss::IOReader *)
